In [22]:
# My Second PySpark Code using Jupyter Notebook

'''

In this program, I try and retrieve data of two tables from MySQL to PySpark, 
perform a join and then load the dataset into a new table in MySQL

'''

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [23]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['SPARK_HOME'] = sys.executable


from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('JoiningTwoTablesCode').config("spark.jars", "mysql-connector-j-8.1.0").getOrCreate()

In [7]:
# Importing the 'actor' table from the database into the actor Spark dataframe

actor = spark.read.format("jdbc").option("driver","com.mysql.cj.jdbc.Driver").option("url", "jdbc:mysql://localhost:3306/sakila").option("dbtable", "actor").option("user", "root").option("password", "Usha@789").load()

In [8]:
# Importing the 'film_actor' table from the database into the film_actor Spark dataframe


film_actor = spark.read.format("jdbc").option("driver","com.mysql.cj.jdbc.Driver").option("url", "jdbc:mysql://localhost:3306/sakila").option("dbtable", "film_actor").option("user", "root").option("password", "Usha@789").load()

In [9]:
# Importing the 'film' table from the database into the film Spark dataframe

film = spark.read.format("jdbc").option("driver","com.mysql.cj.jdbc.Driver").option("url", "jdbc:mysql://localhost:3306/sakila").option("dbtable", "film").option("user", "root").option("password", "Usha@789").load()

In [19]:
# Display the first row of the actor dataframe

actor.show(1)

+--------+----------+---------+-------------------+
|actor_id|first_name|last_name|        last_update|
+--------+----------+---------+-------------------+
|       1|  PENELOPE|  GUINESS|2006-02-15 04:34:33|
+--------+----------+---------+-------------------+
only showing top 1 row



In [20]:
# Display the first row of the film_actor dataframe

film_actor.show(1)

+--------+-------+-------------------+
|actor_id|film_id|        last_update|
+--------+-------+-------------------+
|       1|      1|2006-02-15 05:05:03|
+--------+-------+-------------------+
only showing top 1 row



In [21]:
# Display the first row of the film dataframe

film.show(1)

+-------+----------------+--------------------+------------+-----------+--------------------+---------------+-----------+------+----------------+------+--------------------+-------------------+
|film_id|           title|         description|release_year|language_id|original_language_id|rental_duration|rental_rate|length|replacement_cost|rating|    special_features|        last_update|
+-------+----------------+--------------------+------------+-----------+--------------------+---------------+-----------+------+----------------+------+--------------------+-------------------+
|      1|ACADEMY DINOSAUR|A Epic Drama of a...|  2006-01-01|          1|                NULL|              6|       0.99|    86|           20.99| PG   |Deleted Scenes,Be...|2006-02-15 05:03:42|
+-------+----------------+--------------------+------------+-----------+--------------------+---------------+-----------+------+----------------+------+--------------------+-------------------+
only showing top 1 row



In [12]:
# Working with SQL

# Creating temporary views on the above dataframes to run SQL queries

actor.createOrReplaceTempView("actor_view")
spark.sql("SELECT count(*) from actor_view").show()

film_actor.createOrReplaceTempView("film_actor_view")
spark.sql("SELECT count(*) from film_actor_view").show()

film.createOrReplaceTempView("film_view")
spark.sql("SELECT count(*) from film_view").show()

+--------+
|count(1)|
+--------+
|     200|
+--------+

+--------+
|count(1)|
+--------+
|    5462|
+--------+

+--------+
|count(1)|
+--------+
|    1000|
+--------+



In [17]:
# join_three_tables dataframe will contain data after joining three temporary views using SQL

join_three_tables = spark.sql("SELECT AV.first_name,AV.last_name,FV.title,FV.description,FV.rating from actor_view AV INNER JOIN film_actor_view FAV ON FAV.actor_id = AV.actor_id INNER JOIN film_view FV ON FV.film_id = FAV.film_id")

In [18]:
# This will create a table named actor_film with the data stored in join_three_tables dataframe

join_three_tables.write.format("jdbc").option("driver","com.mysql.cj.jdbc.Driver").option("url", "jdbc:mysql://localhost:3306/sakila").option("dbtable", "actor_film").option("user", "root").option("password", "Usha@789").save()